In [2]:
import os

import ipywidgets as widgets
from IPython.display import display

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer, TFBertForSequenceClassification
from tqdm.notebook import tqdm
import numpy as np
from keras.utils import np_utils
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback 
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tensorflow.keras.metrics import FalseNegatives, FalsePositives, TrueNegatives, TruePositives
from numba import cuda 
import mlflow
import mlflow.tensorflow

mlflow.set_tracking_uri("http://webengineering.ins.hs-anhalt.de:41004")
mlflow.set_experiment("SMART21: Literal Classifier")
mlflow.tensorflow.autolog()
    
strategy = tf.distribute.get_strategy()
device = cuda.get_current_device()

In [3]:
test_data_name = widgets.RadioButtons(
    options=['DBpedia', 'Wikidata'],
    description='Test dataset:',
    disabled=False
)
display(test_data_name)

RadioButtons(description='Test dataset:', options=('DBpedia', 'Wikidata'), value='DBpedia')

In [4]:
external_data = widgets.SelectMultiple(
    options=['DBpedia', 'Wikidata'],
    value=['DBpedia', 'Wikidata'],
    #rows=10,
    description='External Data:',
    disabled=False
)
display(external_data)

SelectMultiple(description='External Data:', index=(0, 1), options=('DBpedia', 'Wikidata'), value=('DBpedia', …

In [5]:
model = widgets.RadioButtons(
    options=['bert-base-cased'],
    description='Model:',
    disabled=False
)
display(model)

RadioButtons(description='Model:', options=('bert-base-cased',), value='bert-base-cased')

In [6]:
epochs = widgets.IntSlider(
    value=10,
    min=1,
    max=20,
    step=1,
    description='EPOCHS:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(epochs)

IntSlider(value=10, continuous_update=False, description='EPOCHS:', max=20, min=1)

In [7]:
MODEL = model.value
EPOCHS = epochs.value

In [8]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [9]:
def regular_encode(texts, tokenizer, maxlen=512):
    """
    encodes text for a model
    """
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [10]:
def build_model(transformer, max_len=512, hidden_dim=32, n_classes=1):
    """
    builds a model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    
    if n_classes == 2: # binary classification
        out = Dense(1, activation='sigmoid')(cls_token)
    else:
        out = Dense(n_classes, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    
    if n_classes > 2:
        model.compile(
            Adam(lr=1e-5),
            loss='categorical_crossentropy',
            metrics=['categorical_accuracy']
        )
    else:
        model.compile(
            Adam(lr=1e-5),
            loss='binary_crossentropy',
            metrics=['accuracy', FalseNegatives(), FalsePositives(), TrueNegatives(), TruePositives()]
        )
    
    return model

In [11]:
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

## Prepare Data

In [13]:
dbp_df = pd.read_csv("../../data/at/dbpedia/task1_dbpedia_train_cleaned.csv", sep='$')
wdt_df = pd.read_csv("../../data/at/wikidata/task1_wikidata_train_cleaned.csv", sep='$')

dbp_df.id = dbp_df.id.apply(lambda x: str(x) + "-dbp")
wdt_df.id = dbp_df.id.apply(lambda x: str(x) + "-wdt")

dbp_df = dbp_df[dbp_df.category == 'literal']
wdt_df = wdt_df[wdt_df.category == 'literal']

data_dict = {
    'DBpedia': dbp_df,
    'Wikidata': wdt_df
}

In [14]:
def add_external_data(df):
    for k in external_data.value:
        if k != test_data_name.value:
            df = df.append(data_dict[k])
    
    return df

In [15]:
max_len = list()

for q in add_external_data(data_dict[test_data_name.value]).question.values:
    max_len.append(len(tokenizer.encode(q)))
    
MAX_LEN = np.array(max_len).max()
del max_len

In [16]:
n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
skf.get_n_splits(data_dict[test_data_name.value].question, data_dict[test_data_name.value].type)

3

In [17]:
train_list = []
test_list = []

for train_index, test_index in skf.split(data_dict[test_data_name.value].id, data_dict[test_data_name.value].type):
    train_df = add_external_data(data_dict[test_data_name.value].iloc[train_index])
    test_df = data_dict[test_data_name.value].iloc[test_index]
    train_df = train_df[~train_df.question.isin(test_df.question)]
    train_list.append(train_df)
    test_list.append(test_df)

In [18]:
test_list[0].shape, train_list[0].shape

((1406, 4), (6338, 4))

In [19]:
tf_train_list, tf_test_list, lens, y_test_list = list(), list(), list(), list()

encoder = LabelEncoder()
encoder.fit(train_list[0].type.values.tolist() + test_list[0].type.values.tolist())
    
for i in range(n_splits):
    x_train = regular_encode(train_list[i].question.values.tolist(), tokenizer, maxlen=MAX_LEN)
    x_test = regular_encode(test_list[i].question.values.tolist(), tokenizer, maxlen=MAX_LEN)

    y_train = train_list[i].type.values
    y_test = test_list[i].type.values

    # encode textual labels into corresponding numbers
    
    encoded_y_train = encoder.transform(y_train) 
    encoded_y_test = encoder.transform(y_test)
    dummy_y_train = np_utils.to_categorical(encoded_y_train, num_classes=encoder.classes_.shape[0]) # one hot encoded
    dummy_y_test = np_utils.to_categorical(encoded_y_test, num_classes=encoder.classes_.shape[0]) # one hot encoded
    
    train_dataset = (
        tf.data.Dataset
        .from_tensor_slices((x_train, dummy_y_train))
        .repeat()
        .shuffle(2048)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    test_dataset = (
        tf.data.Dataset
        .from_tensor_slices((x_test, dummy_y_test))
        .batch(BATCH_SIZE)
    )
    
    tf_train_list.append(train_dataset)
    tf_test_list.append(test_dataset)
    lens.append(x_train.shape)
    y_test_list.append(encoded_y_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ins-alex/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2126: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
2021-08-19 17:47:05.587102: I tensorflow/stream_executor/platform/default/dso_loader.cc:5

## Train Model

In [20]:
split_idx = widgets.IntSlider(
    value=0,
    min=0,
    max=n_splits-1,
    step=1,
    description='N_SPLIT:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(split_idx)

IntSlider(value=0, continuous_update=False, description='N_SPLIT:', max=2)

In [ ]:
i = split_idx.value

with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN, n_classes=encoder.classes_.shape[0])

print(model.summary())

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    verbose=1,
    patience=1,
    min_delta=0.01,
    restore_best_weights=True
)

with mlflow.start_run():
    mlflow.log_param("EPOCHS", EPOCHS)
    mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
    mlflow.log_param("MAX_LEN", MAX_LEN)
    mlflow.log_param("MODEL", MODEL)
    mlflow.log_param("TEST_DATA", test_data_name.value)
    mlflow.log_param("EXTERNAL_DATA", '+'.join(d for d in external_data.value))
    mlflow.log_param("n_split_idx", i)

    n_steps = lens[0][0] // BATCH_SIZE # determine number of steps per epoch

    train_history = model.fit(
        tf_train_list[i],
        steps_per_epoch=n_steps,
        validation_data=tf_test_list[i],
        callbacks=[early_stopping],
        epochs=EPOCHS
    )

2021-08-19 17:47:15.071276: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-08-19 17:47:15.490355: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


/home/ins-alex/.local/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 145)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  TFBaseModelOutputWithPool 108310272 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 7)                 5383      
Total params: 108,315,655
Trainable params: 108,315,655
Non-trainable params: 0
_________________________________________________________________
None
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possib

2021-08-19 17:47:20.181046: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-19 17:47:20.181119: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-19 17:47:20.181207: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2021-08-19 17:47:20.181789: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-08-19 17:47:20.183398: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcupti.so


...

  1/396 [..............................] - ETA: 1:06:51 - loss: 1.6845 - categorical_accuracy: 0.3125

2021-08-19 17:47:31.136279: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-19 17:47:31.136305: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.


  2/396 [..............................] - ETA: 5:45 - loss: 1.5123 - categorical_accuracy: 0.2500   

2021-08-19 17:47:31.665620: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-19 17:47:31.666300: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1743] CUPTI activity buffer flushed
2021-08-19 17:47:31.719237: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 2833 callback api events and 2858 activity events. 
2021-08-19 17:47:31.769271: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-08-19 17:47:31.839323: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /tmp/tmp9yqnj2kg/train/plugins/profile/2021_08_19_17_47_31
2021-08-19 17:47:31.882059: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /tmp/tmp9yqnj2kg/train/plugins/profile/2021_08_19_17_47_31/ins-alex-ThinkPad-T15g-Gen-1.trace.json.gz
2021-08-19 17:47:31.975152: I tensorflow/core/profiler/rpc/client/save_profile.cc

396/396 [==============================] - 156s 370ms/step - loss: 0.3831 - categorical_accuracy: 0.8333 - val_loss: 0.1451 - val_categorical_accuracy: 0.9552
Epoch 2/10
363/396 [==========================>...] - ETA: 11s - loss: 0.1116 - categorical_accuracy: 0.9657